In [1]:
import numpy as np
import pandas as pd
import json

In [18]:
def flatten_data_for_BERT(file_directory):

    with open(file_directory, 'r') as file:
        data = json.load(file)

    flattened_data = []

    # iterate through each entry in the data
    for entry in data:
        question_id = entry['id']
        ambiguous_question = entry['question']

        # check the type of annotations and process accordingly
        for annotation in entry['annotations']:
            if annotation['type'] == 'multipleQAs':
                flattened_data.append({
                    'ambiguous_question': ambiguous_question,
                    'ambig_class': 1
                })
                # for qa_pair in annotation['qaPairs']:
                #     flattened_data.append({
                #         'ambiguous_question': qa_pair['question'],
                #         'ambig_class': 0
                #     })
            elif annotation['type'] == 'singleAnswer':
                flattened_data.append({
                    'ambiguous_question': ambiguous_question,
                    'ambig_class': 0
                })

    ambiguous_df = pd.DataFrame(flattened_data)
    return ambiguous_df

In [19]:
ambiguous_train_BERT_df = flatten_data_for_BERT('../../data/LIGHT/train_light.json')
ambiguous_train_BERT_df = ambiguous_train_BERT_df.dropna()
print(len(ambiguous_train_BERT_df))
ambiguous_train_BERT_df.head()

10251


,ambiguous_question,ambig_class
0,When did the simpsons first air on television?,1
1,Who played george washington in the john adams...,0
2,What is the legal age of marriage in usa?,1
3,Who starred in barefoot in the park on broadway?,1
4,When did the manhattan project began and end?,1


In [20]:
num_0s = len(ambiguous_train_BERT_df.loc[ambiguous_train_BERT_df['ambig_class']==0])
num_1s = len(ambiguous_train_BERT_df.loc[ambiguous_train_BERT_df['ambig_class']==1])
print("Number of unambiguous questions: ", num_0s)
print("Number of ambiguous questions: ", num_1s)

Number of unambiguous questions:  5459
Number of ambiguous questions:  4792


In [21]:
ambiguous_dev_BERT_df = flatten_data_for_BERT('../../data/LIGHT/dev_light.json')
ambiguous_dev_BERT_df = ambiguous_dev_BERT_df.dropna()
ambiguous_dev_BERT_df.head()

,ambiguous_question,ambig_class
0,Who plays the doctor in dexter season 1?,0
1,How often does spermatogeneis—the production o...,0
2,How often does spermatogeneis—the production o...,0
3,When was the first remote control tv invented?,0
4,When was the first remote control tv invented?,0


In [22]:
num_0s = len(ambiguous_dev_BERT_df.loc[ambiguous_dev_BERT_df['ambig_class']==0])
num_1s = len(ambiguous_dev_BERT_df.loc[ambiguous_dev_BERT_df['ambig_class']==1])
print("Number of unambiguous questions: ", num_0s)
print("Number of ambiguous questions: ", num_1s)

Number of unambiguous questions:  1776
Number of ambiguous questions:  1536


In [23]:
# Calculate the index to split the DataFrame into two parts
halfway_idx = len(ambiguous_dev_BERT_df) // 2
print(halfway_idx)
ambiguous_dev_BERT_full_df = ambiguous_dev_BERT_df

# Split the DataFrame into two halves
ambiguous_test_BERT_df = ambiguous_dev_BERT_full_df.iloc[:halfway_idx]
ambiguous_dev_BERT_df = ambiguous_dev_BERT_full_df.iloc[halfway_idx:]
      
print(len(ambiguous_test_BERT_df))
print(len(ambiguous_dev_BERT_df))

1656
1656
1656


In [8]:
ambiguous_train_BERT_df.to_csv('../../data/BERT/ambig_train_BERT.csv', index=False)
ambiguous_dev_BERT_df.to_csv('../../data/BERT/ambig_dev_BERT.csv', index=False)
ambiguous_test_BERT_df.to_csv('../../data/BERT/ambig_test_BERT.csv', index=False)
ambiguous_dev_BERT_full_df.to_csv('../../data/BERT/ambig_dev_BERT_full.csv', index=False)